[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/basantab/DeltaDihedral/blob/main/CaDeltaDihedeal.ipynb)


### Download requirements


In [ ]:
!pip install ProDy
!pip install seaborn

### Calculate and plot values

In [ ]:
import prody as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Download PDB files from open and closed Ec MBP
pdb1ANF = pd.fetchPDB('1ANF')
pdb1OMP = pd.fetchPDB('1OMP')

# Parse PDBs into prody opbjects
obj1ANF =pd.parsePDB(pdb1ANF)
obj1OMP =pd.parsePDB(pdb1OMP)

# Get all Calpha atoms
CA1OMP = obj1OMP.select('chain A and name CA')
CA1ANF = obj1ANF.select('chain A and name CA')

CA1OMP_dict = {r:i for i,r in enumerate(CA1OMP.getResnums())}
CA1ANF_dict = {r:i for i,r in enumerate(CA1ANF.getResnums())}

# Keep only residues in common in both structures
common_res = np.intersect1d(CA1OMP.getResnums(),CA1ANF.getResnums())
#print(common_res)


def norm(ang):
  '''This function normalizes delta angles the same way as Marvin et al'''
  if ang >=-180 and ang <= 180: return ang
  if ang > 180 : return 360 - ang
  if ang < -180: return 360 + ang 

def get_dihedral_alaMarvin(pos,pdb):
  '''This function calculates dihedral CA angles the same way as Marvin et al'''
  return pd.calcDihedral(pdb[pos+2],pdb[pos+1],pdb[pos],pdb[pos-1])

deltas = [] # We are going to store all dihedrals here while we loop below

positions = np.array(common_res[1:-3]) # Because of how the dihedrals are calculated the first and two last residues don't have an assigned angle

# Loop through positions and get the change in CA dihedral
for pos in positions:
  delta = get_dihedral_alaMarvin(CA1ANF_dict[pos],CA1ANF) - get_dihedral_alaMarvin(CA1OMP_dict[pos],CA1OMP)
  #print(pos,norm(delta))
  deltas.append(delta)

# Plot results by position:
sns.set(font_scale=1.5)
plt.figure(figsize = (15,8))
p = sns.regplot(x=positions, y=np.array([norm(i) for i in deltas]),fit_reg=False)
p.set_xlabel("Amino acid", fontsize = 20)
p.set_ylabel("Change in Ca Dihedral (º)", fontsize = 20)


### Print out values

In [ ]:
print("Position DihedralChange")
for i,j in zip(positions,[norm(i) for i in deltas]):
  print(i,j)